In [1]:
from ladle.ladle import Ladle
from dataclasses import dataclass
import requests
from io import BytesIO
import fitz  # PyMuPDF
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [2]:
def extract_pdf_content(doc_url):
    """
    Extract PDF content using PyMuPDF (fitz) - handles complex layouts well
    """
    pdf_resp = requests.get(doc_url)
    pdf_bytes = pdf_resp.content
    
    pdf_text = None
    try:
        # Open PDF from bytes
        pdf_doc = fitz.open(stream=pdf_bytes, filetype="pdf")
        
        # Extract text from all pages
        text_pages = []
        for page_num in range(pdf_doc.page_count):
            page = pdf_doc[page_num]
            # Extract text - this preserves layout better than PyPDF2
            page_text = page.get_text()
            if page_text.strip():  # Only add non-empty pages
                text_pages.append(page_text)
                
        # Join all pages
        pdf_text = "\n\n".join(text_pages) if text_pages else None
        # Close the document
        pdf_doc.close()
        
    except Exception as e:
        print(f"Failed to extract text from PDF: {e}")
        pdf_text = None
    
    return {'pdf_text': pdf_text, 'pdf_bytes': pdf_bytes}

In [3]:
"""
1. Norme e leggi
https://www.parlamento.it/Parlamento/519
https://www.giustizia-amministrativa.it/il-codice-del-processo-amministrativo1,
https://www.giustizia-amministrativa.it/web/guest/codice-dei-contratti-pubblici-approvato-con-d.lgs.-31-marzo-2023-n.-36,
# Plus: link diretti alle fonti delle leggi individuate

2. Risoluzioni, Circolari e Provvedimenti del Direttore dell'Agenzia delle Entrate
https://www.agenziaentrate.gov.it/portale/normativa-e-prassi/risoluzioni/archivio-risoluzioni
https://www.agenziaentrate.gov.it/portale/web/guest/normativa-e-prassi/circolari/archivio-circolari
https://www.agenziaentrate.gov.it/portale/web/guest/archivio/normativa-prassi-archivio-documentazione/provvedimenti/provvedimenti-soggetti
https://www.agenziaentrate.gov.it/portale/archivio/normativa-prassi-archivio-documentazione/provvedimenti/altri-provvedimenti-non-soggetti

3. Commenti
# Piattaforma Valore24 o Eutekne o others

4. Interpelli
https://www.agenziaentrate.gov.it/portale/normativa-e-prassi/risposte-agli-interpelli/interpelli/archivio-interpelli
https://www.agenziaentrate.gov.it/portale/web/guest/archivio-istanze-di-interpello-sui-nuovi-investimenti
https://www.agenziaentrate.gov.it/portale/normativa-e-prassi/risposte-agli-interpelli/principi-di-diritto/archivio-principi-di-diritto
https://www.agenziaentrate.gov.it/portale/normativa-e-prassi/risposte-agli-interpelli/risposte-alle-istanze-di-consulenza-giuridica/archivio-risposte-alle-istanze-di-consulenza-giuridica

5. Sentenze
https://www.italgiure.giustizia.it/sncass/ (corte cassaz.)

6. Altre fonti
https://www.odcec.mi.it/aree-tematiche/formazione/quaderni
https://www.odcec.mi.it/lordine/centro-studi-odcec-milano/2
"""

"""
Fonti in ordine di importanza

1 Norme e leggi (sito del parlamento, parte fiscale codice civile, testo unico)
2 risoluzioni (risposte ufficiali di agenzia delle entrate - non sono leggi, sono anche in s24h)
3 commenti (libri, pubblicazioni sole24ore con testi di pubblicisti, eutechne)
4 interpelli (agenzia risponde ad una domanda per futuri altri richiedenti) pubblica
5 sentenze (cassazione, livello provinciale o regionale) alla pari degli interpelli, siti a pagamento. Forse agenzia entrate ha obbligo di pubblicare tutte le sentenze (di tipo commissione tributaria)
"""

'\nFonti in ordine di importanza\n\n1 Norme e leggi (sito del parlamento, parte fiscale codice civile, testo unico)\n2 risoluzioni (risposte ufficiali di agenzia delle entrate - non sono leggi, sono anche in s24h)\n3 commenti (libri, pubblicazioni sole24ore con testi di pubblicisti, eutechne)\n4 interpelli (agenzia risponde ad una domanda per futuri altri richiedenti) pubblica\n5 sentenze (cassazione, livello provinciale o regionale) alla pari degli interpelli, siti a pagamento. Forse agenzia entrate ha obbligo di pubblicare tutte le sentenze (di tipo commissione tributaria)\n'

In [4]:
ladle = Ladle(headless=False)

In [8]:
ladle.driver.get("https://www.agenziaentrate.gov.it/portale/web/guest/archivio/normativa-prassi-archivio-documentazione/provvedimenti/provvedimenti-soggetti")
try:
    ladle.clicks.click('//*[@id="closePopup"]')
except Exception as e:
    print(f"Site opened with no popup showing")

Site opened with no popup showing


In [15]:
ladle.driver.get("https://www.agenziaentrate.gov.it/portale/web/guest/archivio/normativa-prassi-archivio-documentazione/provvedimenti/provvedimenti-soggetti")
try:
    ladle.clicks.click('//*[@id="closePopup"]')
except Exception as e:
    print(f"Site opened with no popup showing")

elements = ladle.elements.elements('/html/body/div[1]/div/div/div/div[3]/div/div/div[3]/main/div[*]/div/div/section/div/div[2]/div/div/div/div/p/a')
for i in range(len(elements)):
    # Save current page URL to return after visiting subsection. Years page.
    last_page_url = ladle.driver.current_url
    ladle.clicks.click(f'/html/body/div[1]/div/div/div/div[3]/div/div/div[3]/main/div[{2+i}]/div/div/section/div/div[2]/div/div/div/div/p/a')

    # Re-capture elements on the new page (avoid stale references)
    elements = ladle.elements.elements('/html/body/div[4]/div/div/div/div[3]/div/div/div[3]/main/div[2]/div/div/section/div/div[2]/div/div[*]/p/a')

    for j in range(len(elements)):
        # Save current inner page URL to return after visiting detail link. Months page.
        inner_last_page_url = ladle.driver.current_url
        # Div changes dynamically from 4 to 1
        try:
            ladle.clicks.click(f"/html/body/div[4]/div/div/div/div[3]/div/div/div[3]/main/div[2]/div/div/section/div/div[2]/div/div[{1+j}]/p/a")
        except Exception as e:
            ladle.clicks.click(f"/html/body/div[1]/div/div/div/div[3]/div/div/div[3]/main/div[2]/div/div/section/div/div[2]/div/div[{1+j}]/p/a")

        # Document harvesting
        elements = ladle.elements.elements('//*[@id="portlet_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_bpfu"]/div/div[2]/div/div[*]/div/p/a')
        for h in range(len(elements)):

            doc_xpath = f'//*[@id="portlet_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_bpfu"]/div/div[2]/div/div[{1+h}]/div/p/a'
            doc_elem = ladle.elements.element(doc_xpath)
            
            if ladle.elements.element(doc_xpath).text.endswith('pdf'):
                doc_url = ladle.elements.element(doc_xpath).get_attribute('href')
                pdf_content = extract_pdf_content(doc_url)

                # assemble python object
                pdf_obj = {
                    "url": doc_url,
                    "bytes": pdf_content['pdf_bytes'],
                    "text": pdf_content['pdf_text']
                }
                # Load the doc to gcp storage
                # TODO: implement upload to GCP
            else:
                print(f"{doc_elem.text} is not a PDF, stepping into folder...")

                # Save current doc list page URL to return to after visiting the folder
                doc_list_page_url = ladle.driver.current_url

                ladle.clicks.click(doc_xpath)

                # Explore folder contents
                elements = ladle.elements.elements('//*[@id="portlet_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_enri"]/div/div[2]/div/div/div[2]/div/div/ul/li[*]/a')
                for k in range(len(elements)):
                    # Save current folder_doc list page URL to return to after visiting the docs into the clicked folder
                    folder_doc_list_page_url = ladle.driver.current_url

                    folder_doc_xpath = f'//*[@id="portlet_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_enri"]/div/div[2]/div/div/div[2]/div/div/ul/li[{1+k}]/a'
                    folder_doc_elem = ladle.elements.element(folder_doc_xpath)
                    if folder_doc_elem.text.endswith('pdf'):
                        folder_doc_url = folder_doc_elem.get_attribute('href')
                        folder_pdf_content = extract_pdf_content(folder_doc_url)

                        # assemble python object
                        folder_pdf_obj = {
                            "url": folder_doc_url,
                            "bytes": folder_pdf_content['pdf_bytes'],
                            "text": folder_pdf_content['pdf_text']
                        }
                        # Load the doc to gcp storage
                        # TODO: implement upload to GCP
                    else:
                        print(f"{folder_doc_elem.text} is not a PDF, skipping...")
                        # ladle.clicks.click(folder_doc_xpath)
                        # Don't do nothing (for now). Already on the folder doc list page if i don't click.
                        # ladle.driver.get(folder_doc_list_page_url)
                
                # Return to doc list page
                ladle.driver.get(doc_list_page_url)

        # Return to inner list using URL instead of back
        ladle.driver.get(inner_last_page_url)

    # Return to outer list using URL instead of back
    ladle.driver.get(last_page_url)

Site opened with no popup showing
Modalità e termini di comunicazione delle opzioni per l’applicazione dell’imposta sostitutiva per annualità ancora accertabili per i soggetti che aderiscono al concordato preventivo biennale is not a PDF, stepping into folder...
Modalità e termini di comunicazione delle opzioni per l’applicazione dell’imposta sostitutiva per annualità ancora accertabili per i soggetti che aderiscono al concordato preventivo biennale is not a PDF, stepping into folder...
Approvazione del modello di notifica per l’individuazione del soggetto tenuto a presentare la comunicazione rilevante, di cui all’articolo 3 del decreto del Vice Ministro dell’Economia e delle finanze del 25 febbraio 2025, e definizione delle relative modalità di trasmissione is not a PDF, stepping into folder...
Approvazione del modello di notifica per l’individuazione del soggetto tenuto a presentare la comunicazione rilevante, di cui all’articolo 3 del decreto del Vice Ministro dell’Economia e delle 

ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (784, 662)
  (Session info: chrome=140.0.7339.128); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#elementclickinterceptedexception
Stacktrace:
	GetHandleVerifier [0x0xc70c13+66051]
	GetHandleVerifier [0x0xc70c54+66116]
	(No symbol) [0x0xa4db33]
	(No symbol) [0x0xa9dd90]
	(No symbol) [0x0xa9c0f3]
	(No symbol) [0x0xa99ba7]
	(No symbol) [0x0xa98e2d]
	(No symbol) [0x0xa8d335]
	(No symbol) [0x0xab9f8c]
	(No symbol) [0x0xa8cd94]
	(No symbol) [0x0xaba144]
	(No symbol) [0x0xadb7f1]
	(No symbol) [0x0xab9d86]
	(No symbol) [0x0xa8b53e]
	(No symbol) [0x0xa8c414]
	GetHandleVerifier [0x0xeb8a13+2457603]
	GetHandleVerifier [0x0xeb39d2+2437058]
	GetHandleVerifier [0x0xc997f2+232930]
	GetHandleVerifier [0x0xc89a18+167944]
	GetHandleVerifier [0x0xc9092d+196381]
	GetHandleVerifier [0x0xc78ee8+99544]
	GetHandleVerifier [0x0xc79082+

In [ ]:
j = 1


https://www.agenziaentrate.gov.it/portale/2025-provvedimenti-del-direttore-soggetti-a-pubblicita


In [6]:

elements = ladle.elements.elements('//*[@id="portlet_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_bpfu"]/div/div[2]/div/div[*]/div/p/a')
for h in range(len(elements)):
    doc_xpath = f'//*[@id="portlet_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_bpfu"]/div/div[2]/div/div[{1+h}]/div/p/a'
    doc_elem = ladle.elements.element(doc_xpath)
    if ladle.elements.element(doc_xpath).text.endswith('pdf'):
        doc_url = ladle.elements.element(doc_xpath).get_attribute('href')
        pdf_content = extract_pdf_content(doc_url)

        # assemble python object
        pdf_obj = {
            "url": doc_url,
            "bytes": pdf_content['pdf_bytes'],
            "text": pdf_content['pdf_text']
        }
        # Load the doc to gcp storage
        # TODO: implement upload to GCP
    else:
        print(f"{doc_elem.text} is not a PDF, stepping into folder...")
        ladle.clicks.click(doc_xpath)
        # Explore folder contents
        elements = ladle.elements.elements('//*[@id="portlet_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_enri"]/div/div[2]/div/div/div[2]/div/div/ul/li[*]/a')
        for k in range(len(elements)):
            folder_doc_xpath = f'//*[@id="portlet_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_enri"]/div/div[2]/div/div/div[2]/div/div/ul/li[{1+k}]/a'
            folder_doc_elem = ladle.elements.element(folder_doc_xpath)
            if folder_doc_elem.text.endswith('pdf'):
                folder_doc_url = folder_doc_elem.get_attribute('href')
                folder_pdf_content = extract_pdf_content(folder_doc_url)

                # assemble python object
                folder_pdf_obj = {
                    "url": folder_doc_url,
                    "bytes": folder_pdf_content['pdf_bytes'],
                    "text": folder_pdf_content['pdf_text']
                }
                # Load the doc to gcp storage
                # TODO: implement upload to GCP
            else:
                print(f"{folder_doc_elem.text} is not a PDF, skipping...")
                # ladle.clicks.click(folder_doc_xpath)
                # ladle.driver.back()  # Go back to previous page
        ladle.driver.back()
ladle.driver.back()

Approvazione del modello di notifica per l’individuazione del soggetto tenuto a presentare la comunicazione rilevante, di cui all’articolo 3 del decreto del Vice Ministro dell’Economia e delle finanze del 25 febbraio 2025, e definizione delle relative modalità di trasmissione is not a PDF, stepping into folder...
Modello - Notifica per l’individuazione del soggetto tenuto a presentare la comunicazione rilevante is not a PDF, skipping...
Istruzioni per la compilazione is not a PDF, skipping...


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x0xead2a3+66419]
	GetHandleVerifier [0x0xead2e4+66484]
	(No symbol) [0x0xc84bd3]
	(No symbol) [0x0xcce958]
	(No symbol) [0x0xccecfb]
	(No symbol) [0x0xd15152]
	(No symbol) [0x0xcf1064]
	(No symbol) [0x0xd128a1]
	(No symbol) [0x0xcf0e16]
	(No symbol) [0x0xcc25ce]
	(No symbol) [0x0xcc34a4]
	GetHandleVerifier [0x0x10f5ee3+2461619]
	GetHandleVerifier [0x0x10f0f66+2441270]
	GetHandleVerifier [0x0xed6242+234258]
	GetHandleVerifier [0x0xec6208+168664]
	GetHandleVerifier [0x0xecd1ad+197245]
	GetHandleVerifier [0x0xeb55f8+100040]
	GetHandleVerifier [0x0xeb5792+100450]
	GetHandleVerifier [0x0xe9f74a+10266]
	BaseThreadInitThunk [0x0x769d5d49+25]
	RtlInitializeExceptionChain [0x0x77a8d6db+107]
	RtlGetAppContainerNamedObjectPath [0x0x77a8d661+561]


In [8]:
pdf_obj['text']

'_________________________________________________________________________________ \nAgenzia delle Entrate – Direzione Regionale del Lazio – Settore Servizi –Ufficio Servizi Fiscali – \n Via Marcello Boglione 73/81– 00155 Roma – e-mail: dr.lazio.sf@agenziaentrate.it \n \n \n \n \n \nProt. n. 74563 del 11/08/2025 \nAutorizzazione alla società CAF LAVORO S.R.L., C.F. 18008711006, a esercitare \nl’attività di assistenza fiscale nei confronti dei lavoratori dipendenti e pensionati \n \nIL DIRETTORE REGIONALE  \nin base alle attribuzioni conferitegli dalle norme riportate nel seguito del presente \nprovvedimento \nDISPONE \nche la società CAF LAVORO S.R.L., C.F. 18008711006, con sede in Via Angelo Bargoni \nn. 8, 00153, Roma, è autorizzata: \n1. ad esercitare l’attività di assistenza fiscale nei confronti dei lavoratori dipendenti \ne pensionati ai sensi degli articoli 32, comma 1, lettera d), e 34 del d.lgs. 9 luglio \n1997, n. 241; \n2. ad utilizzare la parola “CAF” e “Centro di assistenz

In [ ]:
k = 0
folder_doc_xpath = f'//*[@id="portlet_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_enri"]/div/div[2]/div/div/div[2]/div/div/ul/li[{1+k}]/a'
folder_doc_elem = ladle.elements.element(folder_doc_xpath)
if folder_doc_elem.text.endswith('pdf'):
    folder_doc_url = folder_doc_elem.get_attribute('href')
    folder_pdf_content = extract_pdf_content(folder_doc_url)

    # assemble python object
    folder_pdf_obj = {
        "url": folder_doc_url,
        "bytes": folder_pdf_content['pdf_bytes'],
        "text": folder_pdf_content['pdf_text']
    }
else:
    print(f"{folder_doc_elem.text} is not a PDF, skipping...")
    # ladle.clicks.click(folder_doc_xpath)

In [54]:
# pdf_obj['text']
folder_pdf_obj['text']

'Informativa sul trattamento \ndei dati personali ai sensi \ndegli artt. 13 e 14 del \nRegolamento (UE) 2016/679 \nCon questa informativa l’Agenzia delle entrate spiega come tratta i dati raccolti e quali sono i diritti riconosciuti all’interessato \nai sensi del Regolamento (UE) 2016/679, relativo alla protezione delle persone fisiche con riguardo al trattamento dei dati per\xad\nsonali e del D.Lgs. 196/2003, in materia di protezione dei dati personali. \nFinalità del trattamento\nI dati forniti con questo modello verranno trattati dall’Agenzia delle entrate per le attività connesse alla notifica di cui articolo \n3 del decreto del Vice Ministro dell’economia e delle finanze 25 febbraio 2025, da trasmettere al fine di individuare il soggetto \ntenuto alla presentazione della comunicazione rilevante per conto delle imprese localizzate nel territorio dello Stato italiano e \ndelle entità apolidi costituite in base alla legge dello Stato italiano, rientranti nell’ambito applicativo dell’

In [ ]:
@dataclass
class DocumentRecord:
    source_type: str
    doc_id: str
    title: str
    url: str
    collected_at: str
    text_path: str
    meta: Dict[str, Any] = field(default_factory=dict)

    def to_dict(self):
        d = asdict(self)
        return d